In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.subplots as sp
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, matthews_corrcoef

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
EC=pd.read_csv('/content/drive/MyDrive/MScProject/online_shoppers_intention.csv')
EC.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [5]:
variable=['Weekend','Revenue']
for col in variable:
    encoder = LabelEncoder()
    encoder.fit(EC[col])
    print('Column:', col)
    print('Original categories:', encoder.classes_)
    print('Encoded values:', encoder.transform(encoder.classes_))
    print('\n')
    EC[col] = encoder.fit_transform(EC[col])

Column: Weekend
Original categories: [False  True]
Encoded values: [0 1]


Column: Revenue
Original categories: [False  True]
Encoded values: [0 1]




In [6]:
EC['Month'] = EC['Month'].map({'Feb': 2, 'Mar': 3, 'May': 5,'June':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12})

In [7]:
MMC = ['Administrative_Duration','Informational_Duration','ProductRelated_Duration','BounceRates','ExitRates','PageValues','SpecialDay']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
EC[MMC] = scaler.fit_transform(EC[MMC])

In [8]:
label_encoder = LabelEncoder()

EC['VisitorType'] = label_encoder.fit_transform(EC['VisitorType'])

EC.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,1.00,1.00,0.0,0.0,2,1,1,1,1,2,0,0
1,0,0.0,0,0.0,2,0.001000,0.00,0.50,0.0,0.0,2,2,2,1,2,2,0,0
2,0,0.0,0,0.0,1,0.000000,1.00,1.00,0.0,0.0,2,4,1,9,3,2,0,0
3,0,0.0,0,0.0,2,0.000042,0.25,0.70,0.0,0.0,2,3,2,2,4,2,0,0
4,0,0.0,0,0.0,10,0.009809,0.10,0.25,0.0,0.0,2,3,3,1,4,2,1,0


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/MScProject/data.csv'

EC.to_csv(file_path, index=False)

print("DataFrame saved to CSV successfully.")


DataFrame saved to CSV successfully.


In [9]:
X = EC.drop('Revenue', axis=1)
y = EC['Revenue']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

# Make predictions on the testing set
y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]  # Predict probabilities for class 1

# Calculate AUC score
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC Score:", auc)

# Calculate MCC score
mcc = matthews_corrcoef(y_test, y_pred)
print("MCC Score:", mcc)

              precision    recall  f1-score   support

           0       0.91      0.96      0.94      2055
           1       0.74      0.54      0.63       411

    accuracy                           0.89      2466
   macro avg       0.83      0.75      0.78      2466
weighted avg       0.88      0.89      0.88      2466

AUC Score: 0.9182996785479604
MCC Score: 0.5731161589864427


In [ ]:
feature_importances = rf_classifier.feature_importances_
print("Feature Importances:")
for i, importance in enumerate(feature_importances):
    print(f"Feature {i+1}: {importance}")

Feature Importances:
Feature 1: 0.04191496768159155
Feature 2: 0.05930135936319798
Feature 3: 0.01681245596309233
Feature 4: 0.02708439374721816
Feature 5: 0.07222389059409004
Feature 6: 0.09043300777533891
Feature 7: 0.05715231451763533
Feature 8: 0.0901075820675819
Feature 9: 0.379476953406041
Feature 10: 0.0035078008708331176
Feature 11: 0.04179168516434031
Feature 12: 0.0171821477571345
Feature 13: 0.01905495766928636
Feature 14: 0.031096910255811273
Feature 15: 0.03161390594547984
Feature 16: 0.011867571891548617
Feature 17: 0.009378095329778813


In [12]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X, y)
X_trainn, X_testt, y_trainn, y_testt = train_test_split(X_train_resampled, y_train_resampled, test_size=0.2, random_state=42)
rf_classifier.fit(X_trainn, y_trainn)
y_pred = rf_classifier.predict(X_testt)
print(classification_report(y_testt, y_pred))

# Make predictions on the testing set
y_pred_proba = rf_classifier.predict_proba(X_testt)[:, 1]  # Predict probabilities for class 1

# Calculate AUC score
auc = roc_auc_score(y_testt, y_pred_proba)
print("AUC Score:", auc)

# Calculate MCC score
mcc = matthews_corrcoef(y_testt, y_pred)
print("MCC Score:", mcc)

              precision    recall  f1-score   support

           0       0.95      0.91      0.93      2062
           1       0.92      0.96      0.94      2107

    accuracy                           0.93      4169
   macro avg       0.94      0.93      0.93      4169
weighted avg       0.93      0.93      0.93      4169

AUC Score: 0.9822171441829164
MCC Score: 0.8689346683090209


In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy'],
    'class_weight': [None, 'balanced', 'balanced_subsample']

}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Perform Grid Search
grid_search.fit(X_train, y_train)

# Get the best parameters and the best accuracy score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Accuracy Score:", best_score)
rf_classifier = RandomForestClassifier(**best_params, random_state=42)

# Train the model on the entire training dataset (X_train and y_train combined)
rf_classifier.fit(X_train, y_train)

# Make predictions on the test dataset (X_test)
y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))
# Make predictions on the testing set
y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]  # Predict probabilities for class 1

# Calculate MCC score
mcc = matthews_corrcoef(y_test, y_pred)
print("MCC Score:", mcc)


# Calculate AUC score
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC Score:", auc)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Parameters: {'class_weight': None, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
Best Accuracy Score: 0.9064272341930464


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


              precision    recall  f1-score   support

           0       0.91      0.96      0.94      2055
           1       0.73      0.54      0.62       411

    accuracy                           0.89      2466
   macro avg       0.82      0.75      0.78      2466
weighted avg       0.88      0.89      0.88      2466

MCC Score: 0.567840893125959
AUC Score: 0.9261335180350578


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy'],
    'class_weight': [None, 'balanced', 'balanced_subsample']

}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=10, scoring='accuracy', n_jobs=-1)

# Perform Grid Search
grid_search.fit(X_trainn, y_trainn)

# Get the best parameters and the best accuracy score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Accuracy Score:", best_score)
rf_classifier = RandomForestClassifier(**best_params, random_state=42)

# Train the model on the entire training dataset (X_train and y_train combined)
rf_classifier.fit(X_trainn, y_trainn)

# Make predictions on the test dataset (X_test)
y_pred = rf_classifier.predict(X_testt)
print(classification_report(y_testt, y_pred))
# Make predictions on the testing set
y_pred_proba = rf_classifier.predict_proba(X_testt)[:, 1]  # Predict probabilities for class 1

# Calculate AUC score
auc = roc_auc_score(y_testt, y_pred_proba)
print("AUC Score:", auc)

# Calculate MCC score
mcc = matthews_corrcoef(y_testt, y_pred)
print("MCC Score:", mcc)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Parameters: {'class_weight': None, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
Best Accuracy Score: 0.9261774263852265


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


              precision    recall  f1-score   support

           0       0.95      0.90      0.93      2062
           1       0.91      0.95      0.93      2107

    accuracy                           0.93      4169
   macro avg       0.93      0.93      0.93      4169
weighted avg       0.93      0.93      0.93      4169

AUC Score: 0.9806147077061037
MCC Score: 0.8608219184490283


In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy'],
    'class_weight': [None, 'balanced', 'balanced_subsample']
